In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import nltk
import re
import pickle as pkl
from collections import defaultdict, Counter
from html import unescape

In [2]:
#import umap.umap_ as UMAP
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired

/home/lindali/anaconda3/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/lindali/anaconda3/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/lindali/anaconda3/lib/python3.9/site-packages/umap/distances.py:1086: NumbaDeprecationWa

In [3]:
df_sample = pd.read_csv("./data/clean_ref_ics_data_cleaned_lemmatized_sample.csv")
BERT_results = pd.read_csv("./ics_data_bert_original.csv")

## BERT full model

In [169]:
df_sample.columns

Index(['REF impact case study identifier', 'Main panel',
       'Unit of assessment number', 'Summary impact type',
       '1. Summary of the impact', '2. Underpinning research',
       '3. References to the research', '4. Details of the impact',
       '5. Sources to corroborate the impact', 'case_id',
       'cleaned_1. Summary of the impact', 'cleaned_2. Underpinning research',
       'cleaned_3. References to the research',
       'cleaned_4. Details of the impact',
       'cleaned_5. Sources to corroborate the impact',
       'lemmatized_1. Summary of the impact',
       'lemmatized_2. Underpinning research',
       'lemmatized_3. References to the research',
       'lemmatized_4. Details of the impact',
       'lemmatized_5. Sources to corroborate the impact'],
      dtype='object')

In [10]:
cols = [ 'lemmatized_1. Summary of the impact',
       'lemmatized_2. Underpinning research',
       'lemmatized_4. Details of the impact']

cols_full = [ 'cleaned_1. Summary of the impact',
       'cleaned_2. Underpinning research',
       'cleaned_4. Details of the impact']

df_sample['lemmatized_full_text'] = df_sample[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
df_sample['cleaned_full_text'] = df_sample[cols_full].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

In [11]:
with open("./data/support/high_freq_words.txt", "r") as f:
    high_freq_words = f.read()
high_freq_words =  high_freq_words.split("\n")

In [12]:
def remove_high_freq(t):
    t = t.split(" ")
    t = [word for word in t if word not in high_freq_words]
    t = " ".join(t)
    return t
df_sample['lemmatized_full_text_no_high_freq'] = df_sample['lemmatized_full_text'].map(remove_high_freq)

In [182]:
def bert_model(col, df = df_sample):
    representation_model = KeyBERTInspired()
    topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True, representation_model=representation_model)
    topics, probs = topic_model.fit_transform(list(df[col]))
    return topic_model

In [183]:
%%time
full_bert = bert_model('lemmatized_full_text_no_high_freq')

Batches:   0%|          | 0/125 [00:00<?, ?it/s]

2023-06-11 17:01:52,158 - BERTopic - Transformed documents to Embeddings
2023-06-11 17:02:07,448 - BERTopic - Reduced dimensionality
2023-06-11 17:02:08,234 - BERTopic - Clustered reduced embeddings


CPU times: user 19min 28s, sys: 3min 50s, total: 23min 19s
Wall time: 4min 8s


In [192]:
full_bert.get_topic_info()

,Topic,Count,Name
0,-1,1339,-1_curriculum_educational_health_building
1,0,209,0_curatorial_contemporary_curator_exhibit
2,1,184,1_hospital_healthcare_hospice_medical
3,2,122,2_music_musical_musician_orchestra
4,3,95,3_entrepreneurship_entrepreneur_enterprise_ent...
...,...,...,...
67,66,11,66_classroom_educational_curriculum_school
68,67,11,67_merger_merge_market_competition
69,68,11,68_poverty_income_deprivation_livelihood
70,69,10,69_youth_young_teenager_participation


In [14]:
bert_results = df_sample[["case_id", "lemmatized_full_text_no_high_freq"]]
bert_results["BERT_topic_full"] = full_bert.topics_
bert_results["BERT_proba_full"] = [max(i) for i in full_bert.probabilities_]

In [15]:
huge_cluster = bert_results[bert_results.BERT_topic_full == -1]

In [187]:
huge_cluster_bert = bert_model('lemmatized_full_text_no_high_freq', huge_cluster)

Batches:   0%|          | 0/42 [00:00<?, ?it/s]

2023-06-11 17:03:40,720 - BERTopic - Transformed documents to Embeddings
2023-06-11 17:03:45,892 - BERTopic - Reduced dimensionality
2023-06-11 17:03:46,013 - BERTopic - Clustered reduced embeddings


In [189]:
huge_cluster_bert.get_topic_info()

,Topic,Count,Name
0,-1,384,-1_volunteer_rural_individual_audience
1,0,132,0_crime_criminal_victim_police
2,1,100,1_pedagogical_educator_literacy_educational
3,2,98,2_banking_consumer_regulatory_legislation
4,3,93,3_sustainability_infrastructure_sustainable_en...
5,4,60,4_theatre_playwright_audience_performer
6,5,60,5_organisational_corporate_manage_strategic
7,6,42,6_health_exercise_chronic_clinical
8,7,40,7_health_neurodevelopmental_depression_therapy
9,8,31,8_ideology_brexit_diverse_german


In [213]:
bert_results_cluster = huge_cluster[["case_id"]]
bert_results_cluster["BERT_topic_full_cluster"] = [i+71 if i != -1 else -1 for i in huge_cluster_bert.topics_]
bert_results_cluster["BERT_proba_full_cluster"] = [max(i) for i in huge_cluster_bert.probabilities_]

In [214]:
bert_results = bert_results.merge(bert_results_cluster, how = 'left', on = "case_id", copy = False)

In [215]:
bert_results_cleaned = bert_results.copy()

In [216]:

bert_results_cleaned['BERT_proba_full'] = np.where(bert_results_cleaned['BERT_topic_full'] == -1, 
                                                           bert_results_cleaned['BERT_proba_full_cluster'], 
                                                           bert_results_cleaned['BERT_proba_full'])
bert_results_cleaned['BERT_topic_full'] = np.where(bert_results_cleaned['BERT_topic_full'] == -1, 
                                                           bert_results_cleaned['BERT_topic_full_cluster'], 
                                                           bert_results_cleaned['BERT_topic_full'])


In [ ]:
bert_results_cleaned.drop(columns = ['BERT_topic_full_cluster','BERT_proba_full_cluster','lemmatized_full_text_no_high_freq'], inplace = True)

In [222]:
BERT_results = BERT_results.merge(bert_results_cleaned,  how = 'left', on = "case_id", copy = False)

In [224]:
BERT_results.to_csv("ics_data_bert_original_with_full_text.csv", index = False)

In [230]:
keywords = pd.DataFrame.from_records(full_bert.get_topics()).T.reset_index()
keywords.rename({"index":"topic_id"}, axis=1, inplace = True)

keywords_cluster = pd.DataFrame.from_records(huge_cluster_bert.get_topics()).T.reset_index()
keywords_cluster.rename({"index":"topic_id"}, axis=1, inplace = True)
keywords_cluster['topic_id'] = [i+71 if i != -1 else -1 for i in keywords_cluster.topic_id]


In [231]:
keywords = pd.concat([keywords, keywords_cluster],ignore_index=True)

In [233]:
keywords.to_csv("./models/BERT_keywords_full_text.csv", index = False)

In [234]:
full_bert.save("./models/BERT_original_full")
huge_cluster_bert.save("./models/BERT_original_full_huge_cluster")

## Other results output for analysis

In [22]:
full_bert = BERTopic.load("./models/BERT_original_full")
huge_cluster_bert = BERTopic.load("./models/BERT_original_full_huge_cluster")

### Top X probabilities

In [26]:
len(set(huge_cluster_bert.topics_))

26

In [27]:
def get_top_prob(model, top_n = 5):
    all_probs = []
    for row in model.probabilities_:
        topics = list(range(len(set(full_bert.topics_))-1))
        prob_dict = {k:v for k,v in zip(topics, list(row))}
        prob_dict = dict(sorted(prob_dict.items(), key=lambda item: item[1], reverse=True)[:top_n])
        if model == huge_cluster_bert:
            prob = [(k+71, v) if k!=-1 else (k,v) for k, v in prob_dict.items()]
        else:
            prob = [(k,v) for k, v in prob_dict.items()]
        prob = [item for sublist in prob for item in sublist]
                
        prob_keys = ["topic_top", "prob_top"] * top_n
        prob_keys_id = np.repeat(list(range(1,top_n+1)),2)
        prob_keys = [str(i)+str(j) for i,j in zip(prob_keys,prob_keys_id)]
        
        prob = {k:v for k,v in zip(prob_keys, prob)}
        all_probs.append(prob)
    
    return all_probs

In [28]:
full_bert_prob = get_top_prob(full_bert,71)
cluster_prob = get_top_prob(huge_cluster_bert,71)

In [30]:
minus_ones = [i for i, e in enumerate(full_bert.topics_) if e == -1]
len(minus_ones)

1339

In [31]:
len(cluster_prob)

1339

In [32]:
for (index, replacement) in zip(minus_ones, cluster_prob):
    full_bert_prob[index] = replacement

In [33]:
bert_results_top_5 = pd.DataFrame.from_records(full_bert_prob)
bert_results_top_5['case_id'] =  df_sample[["case_id"]]

In [34]:
bert_results_top_5.columns

Index(['topic_top1', 'prob_top1', 'topic_top2', 'prob_top2', 'topic_top3',
       'prob_top3', 'topic_top4', 'prob_top4', 'topic_top5', 'prob_top5',
       ...
       'prob_top67', 'topic_top68', 'prob_top68', 'topic_top69', 'prob_top69',
       'topic_top70', 'prob_top70', 'topic_top71', 'prob_top71', 'case_id'],
      dtype='object', length=143)

In [35]:
bert_results_top_5.to_csv("./ics_data_modelling_all_full_text.csv", index = False)

### Distance matrix

In [16]:
## Distance matrix
full_hierarchical = full_bert.hierarchical_topics(df_sample['lemmatized_full_text_no_high_freq'])

100%|███████████████████████████████████████████| 70/70 [00:50<00:00,  1.37it/s]


In [18]:
full_hierarchical.to_csv("full_bert_model_hierarchy.csv", index = False)

In [23]:
cluster_hierarchical = huge_cluster_bert.hierarchical_topics(huge_cluster['lemmatized_full_text_no_high_freq'])

100%|███████████████████████████████████████████| 24/24 [00:17<00:00,  1.38it/s]


In [25]:
cluster_hierarchical.to_csv("huge_cluster_bert_model_hierarchy.csv", index = False)

In [26]:
from sklearn.metrics.pairwise import cosine_similarity

distance_matrix = cosine_similarity(np.array(full_bert.topic_embeddings_)[1:, :])
labels = (full_bert.get_topic_info().sort_values("Topic", ascending=True).Name)[1:]

In [30]:
distance_full = pd.DataFrame(distance_matrix, columns = labels).set_index(labels)

In [32]:
distance_full.to_csv("full_bert_model_without_huge_cluster_distance_matrix.csv")

In [33]:
distance_matrix_cluster = cosine_similarity(np.array(huge_cluster_bert.topic_embeddings_)[1:, :])
labels = (huge_cluster_bert.get_topic_info().sort_values("Topic", ascending=True).Name)[1:]
distance_cluster = pd.DataFrame(distance_matrix_cluster, columns = labels).set_index(labels)
distance_cluster.to_csv("bert_model_huge_cluster_distance_matrix.csv")